<h1>1. Importing packages</h1>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import gcsfs
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

<h1>2. Loading the dataset</h1>

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'stalwart-veld-364418'
!gcloud config set project {project_id}

In [ ]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://lewagon-jbaccarin-bucket/preproc_dataset/preproc_data.csv /content/preproc_data.csv

In [ ]:
data = pd.read_csv('preproc_data.csv')
data.shape

<h1>3. Splitting and vectorizing</h1>

In [ ]:
y = data['username']
X = data["code_source"]

In [ ]:
lb = LabelEncoder()
y = lb.fit_transform(y)

In [ ]:
# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

<h3>3.1. TF-IDF on text data:</h3>


In [ ]:
tfidf = TfidfVectorizer(binary=True, max_features=4000, token_pattern='([\w]+|[\s]+|.+)') # token_pattern='[^ ()]+' , ngram_range=(1,5)
def tfidf_features(txt, flag):
    if flag == "train":
        x = tfidf.fit_transform(txt)
    else:
        x = tfidf.transform(txt)
    x = x.astype('float64')
    return x 
X_train = tfidf_features(X_train, flag="train").todense()
X_test = tfidf_features(X_test, flag="test").todense()

<h3>3.2. Making Target as Categorical</h3>

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

<h1>4. Building the model</h1>

In [ ]:
y_train.shape[1] #authors in train data

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(1000, kernel_initializer=keras.initializers.he_normal(seed=1), activation='relu', input_dim=X_train.shape[1]))
model.add(keras.layers.Dropout(0.8))
model.add(keras.layers.Dense(1000, kernel_initializer=keras.initializers.he_normal(seed=2), activation='relu'))
model.add(keras.layers.Dropout(0.8))
model.add(keras.layers.Dense(y_train.shape[1], kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=4), activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

<h1>5. Training the model</h1>
<p>Don't forget to turn on GPU</p>

In [ ]:
from tensorflow.keras import callbacks

es = callbacks.EarlyStopping(patience=15, restore_best_weights=True)

history = model.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=512,
                  verbose=1,
                  validation_split=0.2,
                  callbacks=[es])

<h1>6. Plotting learning curves</h1>
Learning curves show us overting/underfiting

In [ ]:
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


<h1>7. Evaluating model and predicting</h1>

In [ ]:
eval = model.evaluate(X_test, y_test)
eval

In [ ]:
predictions_encoded = model.predict(X_test)
predictions_encoded.shape

<h3>7.1. Converting predicted vectors to names of authors</h13>

In [ ]:
predictions = lb.inverse_transform([np.argmax(pred) for pred in predictions_encoded])
predictions

<h1>8. Exporting Pickles</h1>

In [ ]:
pickle.dump(lb, open('./nn_labelenc.pkl', 'wb')) #label encoder
pickle.dump(tfidf, open('./nn_tfidf.pkl', 'wb')) #tfidf
pickle.dump(model, open('./nn_model.pkl', 'wb')) #model

In [ ]:
!gsutil cp nn_tfidf.pkl gs://lewagon-jbaccarin-bucket/models/
!gsutil cp nn_labelenc.pkl gs://lewagon-jbaccarin-bucket/models/
!gsutil cp nn_model.pkl gs://lewagon-jbaccarin-bucket/models/

CommandException: No URLs matched: /tmp/to_upload.txt
